PR-new-logo-orizontal (1).svg
# **Lab02PR. Web Scraping & RegEx Text Analysis**


#### In acest laborator se vor prezenta urmatoarele:
1. Web Scraping
2. Text Analysis folosind RegEx
3. Ethics of Web Scraping

### 1. Web Scraping

Web Scraping este procesul de extragere de date din paginile web.

În primul rând, web scraper-ul va primi una sau mai multe adrese URL de încărcat înainte de scraping. Scraperul încarcă apoi întregul cod HTML pentru pagina în cauză. Scraperele mai avansate vor reda întregul site web, inclusiv elementele CSS și Javascript.

În mod ideal, utilizatorul va trece prin procesul de selectare a datelor specifice pe care le dorește de pe pagină. De exemplu, este posibil să doriți să extrageti o pagină de produse Amazon pentru prețuri și modele, dar nu sunteți neapărat interesați de recenziile produselor.

În cele din urmă, un web scraper va scoate toate datele care au fost colectate într-un format care este mai util utilizatorului. Majoritatea scraper-urilor web vor scoate date într-un format precum CSV sau Excel, în timp ce scraper-urile mai avansate vor suporta alte formate, cum ar fi JSON.

Pentru partea practica a laboratorului vom face scraping pe site-ul OLX [https://www.olx.ro/](https://www.olx.ro/) de unde vom extrage anunturi de apartamente de inchiriat in Sectorul 6, Bucuresti.

### requests

Primul pas pentru a face scraping este sa importam bibliotexa requests. Aceasta ne va permite sa facem request-uri HTTP.
Vom folosi functia get pentru a face request-ul HTTP pentru pagina pe care o dorim sa o extragem.


In [12]:
import requests

r = requests.get('grzv1.html')
with open("grzv1.html", "w") as out_file:
  out_file.write(r.text)

MissingSchema: Invalid URL 'grzv1.html': No scheme supplied. Perhaps you meant https://grzv1.html?

#### Atributele raspunsului HTTP - [requests.Response](https://requests.readthedocs.io/en/latest/api/#requests.Response)

```python
r.status_code - codul de stare HTTP
r.encoding - encodingul raspunsului
r.headers - antetul raspunsului HTTP
r.content - continutul raspunsului HTTP
r.text - continutul raspunsului HTTP ca string
r.json() - continutul raspunsului HTTP ca JSON (daca raspunsul nu este bine format, va fi generata o exceptie)
```

In [7]:
print("Status code:\n", r.status_code)
print("Headers:\n", r.headers)
# print("Content:\n", r.content)
# print("Text:\n", r.text)
print("Encoding:\n", r.encoding)

Status code:
 200
Headers:
 {'Content-Type': 'text/html; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Date': 'Thu, 19 Oct 2023 17:03:27 GMT', 'Server': 'nginx', 'Referrer-Policy': 'strict-origin-when-cross-origin', 'Content-Security-Policy': "default-src self *  ;style-src  https: data: 'unsafe-inline';img-src  https: blob: data:;child-src data:;object-src none;worker-src blob: https://*.olx.ro  ;frame-src  https: blob:;script-src  https: 'unsafe-inline' 'unsafe-eval';font-src data: self https: ;connect-src self * blob:", 'Cross-Origin-Resource-Policy': 'cross-origin', 'X-Permitted-Cross-Domain-Policies': 'none', 'X-DNS-Prefetch-Control': 'off', 'Expect-CT': 'max-age=0', 'X-Frame-Options': 'SAMEORIGIN', 'Strict-Transport-Security': 'max-age=15552000; includeSubDomains', 'X-Download-Options': 'noopen', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Permissions-Policy': 'accelerometer=(), ambient-light-sensor=(), autoplay=(), battery=(), ca

#### Deoarece structura si clasele site-ului pe care vrem sa il scrape-uim se poate schimba, in acest laborator vom folosi fisierul **grzv1.html** in care este salvata pagina de la adresa [imobiliare grozavesti](https://www.olx.ro/d/imobiliare/apartamente-garsoniere-de-inchiriat/bucuresti/q-grozavesti/?search%5Bdistrict_id%5D=11&currency=EUR).

### LXML

În continuare, vom folosi biblioteca [lxml.html](https://lxml.de/lxmlhtml.html) pentru a extrage datele din paginile web. Aceasta este o bibliotecă de Python care permite utilizatorilor să parcurgă și să modifice fișiere XML și HTML.


Generarea unui arbore de elemente HTML poate fi efectuat folosind:
- [lxml.html.parse](https://lxml.de/api/lxml.html-module.html#parse) - parseaza un document HTML dintr-un fisier / URL
- [lxml.html.fromstring](https://lxml.de/api/lxml.html-module.html#fromstring) - parseaza un document HTML dintr-un string
- [lxml.html.fragment_fromstring](https://lxml.de/api/lxml.html-module.html#fragment_fromstring) - returneaza un fragment de HTML dintr-un string


In [1]:
html_data = open('./grzv1.html', 'r', encoding='utf-8').read()

In [2]:
import lxml.html

tree = lxml.html.fromstring(html_data)

#### Atributele elementelor HTML - [lxml.html.HtmlElement](https://lxml.de/api/lxml.html.HtmlElement-class.html)

```python
element.tag - numele tag-ului

element.text - continutul text al elementului

element.attrib - dictionar cu atributele elementului

element.tail - textul care urmeaza dupa tagul de inchidere al elementului
```

In [3]:
print(type(tree))
print(tree.tag)
print(tree.text)
print(tree.attrib)
print(tree.tail)

<class 'lxml.html.HtmlElement'>
html
None
{}
None


#### Metodele elementelor HTML - [lxml.html.HtmlElement](https://lxml.de/api/lxml.html.HtmlElement-class.html) (mostenite de la [lxml.etree.Element](https://lxml.de/api/lxml.etree._Element-class.html)) pe care le vom folosi in acest laborator sunt:

\* De obicei cand facem scraping stim dupa ce ne uitam (tag-uri, clase, atribute). Iteratorii nu vor fi necesari pentru rezolvarea laboratorului.

```python
Metode mostenite lxml.etree.Element:

element.find() - returneaza primul element dupa tag sau path

element.findall() - returneaza toate elementele dupa tag sau path

element.getchildren() - returneaza o lista cu copiii elementului

element.getparent() - returneaza parintele elementului

element.getnext() - returneaza urmatorul element frate

element.getprevious() - returneaza elementul frate anterior

element.getroottree() - returneaza radacina arborelui

element.getiterator() - returneaza un iterator pentru toate elementele din arbore (elementele vor fi returnate depth-first)

element.items() - returneaza o lista cu perechi (atribut, valoare) pentru element

element.iter() - returneaza un iterator pentru toate elementele din arbore (elementele vor fi returnate depth-first)

element.xpath() - returneaza o lista cu elementele care corespund expresiei XPath

Metode specifice lxml.html.HtmlElement:

element.find_class() - returneaza o lista cu elementele care au clasa specificata

element.find_rel_links() - returneaza o lista cu elementele care au atributul rel specificat (<a rel="{rel}">)

element.text_content() - returneaza continutul text al elementului si a tuturor copiilor


#### Pentru a continua exemplul pentru scraping pe OLX, trebuie sa aflam care este tag-ul care contine anunturile de apartamente de inchiriat in Sectorul 6, Bucuresti. Pentru a afla acest lucru, vom folosi functia inspect din browser. Vom folosi metoda specifica lxml.html.HtmlElement find_class pentru a extrage elementele care au clasa specificata.

#### Vom construi un pd.DataFrame cu datele extrase din pagina web.

In [4]:
# extragem o lista cu elementele ce contin anunturile
ad_list =  tree.find_class('css-1sw7q4x')
print(len(ad_list))

54


In [26]:
art_list = []

# iteram prin lista de anunturi
for i, ad in enumerate(ad_list):

    # extragem titlul anuntului
    title = ad.find_class('css-16v5mdi er34gjf0')
    if len(title) > 0:
        title = title[0].text_content()


    # extragem pretul anuntului
    price = ad.find_class('css-10b0gli er34gjf0')
    if len(price) > 0:
        price = re.findall("[0-9,\.]* €",price[0].text_content())[0]
        # price = price[0].text_content()

    # extragem link-ul catre anunt
    link = ad.find_class('css-rc5s2u')
    if len(link) > 0:
        link = link[0].attrib['href']

    # extragem locatia si data anuntului
    location_date = ad.find_class('css-veheph er34gjf0')
    if len(location_date) > 0:
        location_date = location_date[0].text_content()

    # extragem textul cu detalii despre pret negocibil
    negotiable = ad.find_class('css-1vxklie')
    if len(negotiable) > 0:
        negotiable = negotiable[0].text_content()
    else:
        negotiable = "Pretul nu este negociabil"

    # extragem suprafata
    surface = ad.find_class('css-643j0o')
    if len(surface) > 0:
        surface = surface[0].text_content()

    art_list.append({
        'title': title,
        'price': price,
        'link': link,
        'location_date': location_date,
        'negotiable': negotiable,
        'surface': surface
    })

In [27]:
import pandas as pd
art_df = pd.DataFrame(art_list, columns=['title', 'price', 'link', 'location_date', 'negotiable', 'surface'])
art_df.head(30)

,title,price,link,location_date,negotiable,surface
0,[],[],[],[],Pretul nu este negociabil,[]
1,"Particular, ofer spre inchiriere apartament 2 ...",450 €,/d/oferta/particular-ofer-spre-inchiriere-apar...,"Bucuresti, Sectorul 6 - 15 octombrie 2023",Pretul nu este negociabil,50 m²
2,"Apartament 2 camere Plaza Romania, Lujerului, ...",620 €,/d/oferta/apartament-2-camere-plaza-romania-lu...,"Bucuresti, Sectorul 6 - Reactualizat la 15 oct...",Prețul e negociabil,53 m²
3,2 Camere | Lujerului | 57 MP | Decomandat | 3 ...,500 €,/d/oferta/2-camere-lujerului-57-mp-decomandat-...,"Bucuresti, Sectorul 6 - Reactualizat la 15 oct...",Pretul nu este negociabil,60 m²
4,De inchiriat 2 camere - complet mobilat - Aqua...,600 €,https://www.storia.ro/ro/oferta/de-inchiriat-2...,"Bucuresti, Sectorul 6 - Azi la 15:38",Pretul nu este negociabil,60 m²
5,Apartament 2 camere Lux zona Politehnica/Lujer...,499 €,https://www.storia.ro/ro/oferta/apartament-2-c...,"Bucuresti, Sectorul 6 - Azi la 15:29",Pretul nu este negociabil,58 m²
6,Lujerului | 2 Camere | Centrala | Loc De Parc...,570 €,https://www.storia.ro/ro/oferta/lujerului-2-ca...,"Bucuresti, Sectorul 6 - Azi la 15:17",Pretul nu este negociabil,62 m²
7,Închiriez apartament,201 €,/d/oferta/inchiriez-apartament-IDfRSpR.html,"Bucuresti, Sectorul 6 - Azi la 15:03",Pretul nu este negociabil,67 m²
8,Inchiriez Apartament 3 camere Drumul Taberei,353 €,/d/oferta/inchiriez-apartament-3-camere-drumul...,"Bucuresti, Sectorul 6 - Azi la 14:53",Pretul nu este negociabil,60 m²
9,Grozavesti | Regie Residence | Centrala | Ac |...,550 €,https://www.storia.ro/ro/oferta/grozavesti-reg...,"Bucuresti, Sectorul 6 - Azi la 14:36",Pretul nu este negociabil,50 m²


#### Acum avem primul set de informatii despre anunturi. Urmatorul pas este accesam pagina fiecarui anunt pentru a extrage informatii suplimentare - in exercitii.

### 2. Text Analysis folosind RegEx

#### Ce sunt regex-urile si cum functioneaza ?

O expresie regulata este o secventa de caractere speciale ce formeaza un pattern.

O expresie regulata va fi folosita pentru a cauta / a extrage unul sau mai multe string-uri ce respecta pattern-ul respectiv.

#### De ce folosim regex in stiinta datelor ?

+ Dorim sa vedem daca datele contin un anumit pattern

+ Vrem sa curatam datele raw

+ Vrem sa extragem si sa structuram datele raw

#### Caracterele speciale

``^`` - face match la inceputul stringului; este o ancora (anchor)

``$`` - face match la sfarsitul stringului; este o ancora (anchor)

``.`` - face match pe orice caracter in afara de newline

``|`` - A|B face match pe A sau B unde A si B sunt regex-uri

``\`` - folosit pentru escaparea caracterelor speciale

``\d`` - face match pe cifre

``\D`` - face match pe orice NU e cifra

``\s`` - face match pe orice space/newline (' ', '\t', '\n', '\r', '\f', '\v')

``\S`` - face match pe orice NU este space/newline

``\w`` - face match pe orice caracter alfanumeric (a-zA-Z0-9_)

``\W`` - face match pe orice caracter ce NU este alfanumeric

#### Cuantificatori

greedy match = incearca sa potriveasca cat mai mult text

``*`` - face greedy match de 0 sau mai multe ori pe regex-ul precedent acestuia.

``+`` - face greedy match o data sau de mai multe ori pe regex-ul precedent acestuia.

``?`` - face match pe 0 sau 1 repetitii ale regex-ului anterior

``{m}`` - face match pe exact m repetitii ale regex-ului precedent

``{m,n}`` - face greedy match pe intre m si n repetitii ale regex-ului precedent

``{m,n}?`` - face match pe intre m si n repetitii ale regex-ului precedent dar va incerca sa potriveasca cat mai putin text


#### Seturi de caractere

``[]`` - indica un set de caractere

``[abc]`` - face match pe a, b sau c

``[a-z]`` - face match pe un caracter din intervalul a-z (alte exemple ``[0-9]`` ``[A-Z]`` ``[a-zA-Z0-9])

``[^a]`` - face match pe orice caracter diferit de 'a'. Caracterul ``^`` atunci cand este folosit in seturi nu mai are rolul de a face match la inceputul stringului ci de a reprezeta **complementul setului**.

#### Grupuri

Acestea faciliteaza extragerea si structurarea datelor. Impartirea datelor in anumite subgrupuri face utila procesarea acestora pentru folosirea ulterioara in aplicatii de data science.

``()`` - este reprezentarea unui grup iar acesta trebuie sa contina o expresie regulata; rezultatul fiecarei expresii regulate dintr-un grup poate fi accesat folosind metoda ``re.Match.group(index)`` al obiectului returnat.

``(?:)`` - non-capturing group; se face match pe expresia regulata existenta in grup dar aceasta nu poate fi accesata; acest tip de grup este util cand vrem sa facem match pe anumite date inutile ce nu ne intereseaza (poate o structura a datelor).

``(?P<name>)`` - named group; face acelasi lucru ca ``()`` doar ca rezultatele pot fi accesate si folosind ``re.Match.group(name)``; in cazul in care avem o expresie regulata ce contine mai multe grupuri, rezultatul compilarii poate fi reprezentat sub forma de dictionar folosind ``re.Match.groupdict()``


#### Functii din ``re``

Biblioteca ``re`` ofera suport pentru expresii regulate Pearl-like in Python. In aceasta biblioteca erorile de compilare ale expresiilor regulate sunt identificate prin ``re.error``.

Functiile ``re.match()`` si ``re.search()`` din aceasta biblioteca folosesc obiectul ``re.Match`` pentru a determina daca s-a facut match cu pattern-ul dat ca parametru. Aceste obiecte contin atat substringul pe care s-a facut match cat si pozitia acestuia. Ex: ``<re.Match object; span=(0, 1), match='d'>``

#### Functiile utile pe care le vom folosi in aceasta parte sunt:

#### ``re.match(pattern, string, flags=0)``: returneaza un obiect Match daca pattern-ul se potriveste la inceputul stringului. Altfel returneaza None.


#### ``re.search(pattern, string, flags=0)``: returneaza un obiect Match care contine prima aparitie a pattern-ului. Altfel returneaza None.

#### ``re.split(pattern, string, maxsplit=0, flags=0)``: face split stringului la aparitiile pattern-ului.

#### ``re.findall(pattern, string, flags=0)``: returneaza toate aparitiile nesuprapuse ale pattern-ului (din string) intr-o lista.

#### ``re.finditer(pattern, string, flags=0)``: returneaza un iterator de obiecte ``re.Match`` pe toate aparitiile nesuprapuse ale pattern-ului.

\* Daca explicatiile de mai sus nu au fost suficiente:
https://docs.python.org/3/library/re.html

\* Un cheatsheet util: https://www.dataquest.io/wp-content/uploads/2019/03/python-regular-expressions-cheat-sheet.pdf


In [28]:
import re

s = "Ana are mere. Ana are pere. Ana le primeste de la Ion"
pattern = "Ana"

print("String: \n", s)
print("Pattern: \n", pattern, '\n')


# match returneaza daca pattern-ul se potriveste la inceputul stringului
print("Rezultatul folosind finditer():")
m = re.match(pattern, s)
print(m, '\n')

# search returneaza prima aparitie a pattern-ului in string (re.Match)
print("Rezultatul folosind search():")
m = re.search(pattern, s)
print(m, '\n')

# findall returneaza o lista cu toate aparitiile pattern-ului in string
print("Rezultatul folosind findall():")
m = re.findall(pattern, s)
print(m, '\n')

# finditer returneaza un iterator (re.Match) pe toate aparitiile pattern-ului in
# string
print("Rezultatul folosind finditer():")
m = re.finditer(pattern, s)
print(m)
print("Folosim iteratorul pentru a afisa rezultatele:")
for m_iter in m:
  print(m_iter)

String: 
 Ana are mere. Ana are pere. Ana le primeste de la Ion
Pattern: 
 Ana 

Rezultatul folosind finditer():
<re.Match object; span=(0, 3), match='Ana'> 

Rezultatul folosind search():
<re.Match object; span=(0, 3), match='Ana'> 

Rezultatul folosind findall():
['Ana', 'Ana', 'Ana'] 

Rezultatul folosind finditer():
Folosim iteratorul pentru a afisa rezultatele:
<re.Match object; span=(0, 3), match='Ana'>
<re.Match object; span=(14, 17), match='Ana'>
<re.Match object; span=(28, 31), match='Ana'>


In [29]:
# Caractere speciale, cuantificatori si seturi

s = "Ana are 50 de mere. Ana are 100 de pere [easter egg].\
 Colab-ul e o mizerie.\
 Ana il stie pe Ion.\
 Ana le primeste de la Ion"

# facem match pe Ana daca se afla la inceputul stringului
pattern = "^Ana"
m = re.search(pattern, s)
print(m)
# <re.Match object; span=(0, 3), match='Ana'>

# facem match pe Ion daca se afla la sfarsitul stringului
pattern = "Ion$"
m = re.search(pattern, s)
print(m)
# <re.Match object; span=(105, 108), match='Ion'>

# extragem Ana sau Ion din string
pattern = "Ana|Ion"
m = re.findall(pattern, s)
print(m)
# ['Ana', 'Ana', 'Ana', 'Ion', 'Ana', 'Ion']

# extragem toate numerele din string
pattern = "\d+"
m = re.findall(pattern, s)
print(m)
# ['50', '100']

# extragem toate cuvintele cu litera mare
pattern = "[A-Z][a-z]*"
m = re.findall(pattern, s)
print(m)
# ['Ana', 'Ana', 'Colab', 'Ana', 'Ion', 'Ana', 'Ion']

# extragem easter egg-ul cu paranteze
pattern = "\[.*\]"
m = re.findall(pattern, s)
print(m)
# ['[easter egg]']

# extragem easter egg-ul fara paranteze folosind grup
pattern = "\[(.*)\]"
m = re.findall(pattern, s)
print(m)
# ['easter egg']


<re.Match object; span=(0, 3), match='Ana'>
<re.Match object; span=(118, 121), match='Ion'>
['Ana', 'Ana', 'Ana', 'Ion', 'Ana', 'Ion']
['50', '100']
['Ana', 'Ana', 'Colab', 'Ana', 'Ion', 'Ana', 'Ion']
['[easter egg]']
['easter egg']


In [30]:
# Grupuri
# Pentru exemplificare vom construi un pattern prin care extragem email-urile
# si numele User-Agent-ului

s = \
'''From: author@example.com
User-Agent: Thunderbird (X11/20061227)
To: editor@example.com'''

pattern = "(?:From: )" +\
          "(?P<from>\w+@\w+\.\w+)"  +\
          "(?:\s*User-Agent: )" +\
          "(?P<agent>\w+)" +\
          " \((?P<type>[A-Z]\d+)/(?P<number>\d+)\)" +\
          "\s*(?:To: )" +\
          "(?P<to>\w+@\w+\.\w+)"

m = re.search(pattern, s)
print(m.groupdict())


{'from': 'author@example.com', 'agent': 'Thunderbird', 'type': 'X11', 'number': '20061227', 'to': 'editor@example.com'}


``(?:From: )`` -> nu ne intereseaza => non-capturing group

``(?P<from>\w+@\w+\.\w+)`` -> named group care extrage email-ul

``(?:\s*User-Agent: )`` -> nu ne intereseaza => non-capturing group

``(?P<agent>\w+)`` named group care extrage user-agent-ul

`` \((?P<type>[A-Z]\d+)/(?P<number>\d+)\)`` -> extragem tipul si numarul din structura (tip/nr)

``\s*(?:To: )`` -> nu ne intereseaza => non-capturing group

``(?P<to>\w+@\w+\.\w+)`` -> named group care extrage email-ul

### 3. Etica in Web Scraping

**Modul API**

Unele site-uri web au propriile lor API-uri create special pentru extragerea datelor fără a fi nevoie de scraping. Aceasta înseamnă că ați face-o conform regulilor lor; ați fost autorizat să obțineți informațiile. Deci, dacă există un API, utilizați-l în loc să efectuati scraping.

**Respectați roboții.txt**

Cunoscut și ca standard de excludere a roboților, fișierul robots.txt este ceea ce indică software-ul de accesare cu crawlere pe web unde este permis (sau nu) pe site. Aceasta face parte din Robots Exclusion Protocol (REP), care reprezintă un grup de standarde web create ca o modalitate de a reglementa modul în care roboții accesează cu site-urile web.

**Citiți Termenii și Condițiile**

Acesta este principalul mod în care proprietarul site-ului vă spune regulile. Da, este mai ușor să faceți clic pe „Sunt de acord” sau „Accept” și sa va faceti treaba. Amintiți-vă că le-au scris pentru un motiv.

**Fii delicat**

Procesul de scraping poate fi destul de dur pe server, iar scrapingul agresiv poate duce uneori la probleme de funcționalitate, generând o experiență proastă pentru utilizatorii umani. Așadar, scrapingul este ideal facut în afara orelor de vârf. Și nu uitați să distanțați solicitările, astfel încât proprietarul site-ului web să nu vă încurce scraping-ul pentru un atac DDoS.

**Identifica-te**

Administratorul site-ului web poate observa un trafic neobișnuit. Manierele sunt pe primul loc, așa că spune-le cine ești, intențiile tale și cum să te contacteze pentru mai multe întrebări. Puteți face acest lucru adăugând pur și simplu **un string User-Agent cu informațiile dvs.**, astfel încât aceștia să le poată vedea.

**Cere permisiunea**

O oarecare curtoazie umană de bază este întotdeauna apreciată. Ei au ceva ce îți dorești, fii politicos și întreabă înainte de a presupune că informațiile sunt libere pentru tine. Rețineți: datele nu vă aparțin.

**Valorificați conținutul pe care îl păstrați**

Ar trebui să luați doar tipul de conținut de care aveți nevoie. Și aveți întotdeauna un motiv bun pentru a obține conținutul în primul rând. Scopul utilizării datelor este de a crea mai multă valoare, nu de a o duplica.

**Tratați datele cu respect**

Ți s-a dat permisiunea de a prelua conținutul, dar asta nu înseamnă că acum poți acorda acea permisiune altora.

**Dă înapoi când poți**

Acordați credit proprietarului site-ului într-un articol sau pe rețelele de socializare și încercați să atrageți trafic bun înapoi pe site-ul lor.


### Exercitii

1. Pe dataframe-ul art_df: (3p)
    
    a. separati `location_date` in `location` si `date` (hint: folositi regex).
    
    b. transformati coloana `price` in float (hint: folositi regex) - aveti grija la "Pretul este negociabil".

    c. transformati coloana `surface` in float (hint: folositi regex).
    
    d. modificati coloana `negotiable` astfel incat sa contina doar valori 0 sau 1

2. Efectuati scraping pe link-urile (cel putin 3) din dataframe si adaugati noi coloane in dataframe cu informatiile extrase: (4p)
    
    a. an de constructie
    
    b. etaj
    
    c. numar camere
    
    d. persoana fizica sau juridica
    
    e. compartimentare

3. Alegeti-va o pagina web si efectuati scraping pe ea. Creati un dataframe cu informatiile extrase. (3p)

In [31]:
df = art_df

df[['location', 'date']] = df['location_date'].str.split('-', expand=True)
df = df.drop('location_date', axis=1)

df['price'] = pd.to_numeric(df['price'].str.replace('[^0-9.]', '', regex=True), errors='coerce')
df['surface'] = pd.to_numeric(df['surface'].str.replace('[^0-9.]', '', regex=True), errors='coerce')
df['negotiable'] = df['negotiable'].apply(lambda x: 1 if x == 'Prețul e negociabil' else 0)

In [32]:
df.head(20)

,title,price,link,negotiable,surface,location,date
0,[],NaN,[],0,NaN,NaN,NaN
1,"Particular, ofer spre inchiriere apartament 2 ...",450.0,/d/oferta/particular-ofer-spre-inchiriere-apar...,0,50.0,"Bucuresti, Sectorul 6",15 octombrie 2023
2,"Apartament 2 camere Plaza Romania, Lujerului, ...",620.0,/d/oferta/apartament-2-camere-plaza-romania-lu...,1,53.0,"Bucuresti, Sectorul 6",Reactualizat la 15 octombrie 2023
3,2 Camere | Lujerului | 57 MP | Decomandat | 3 ...,500.0,/d/oferta/2-camere-lujerului-57-mp-decomandat-...,0,60.0,"Bucuresti, Sectorul 6",Reactualizat la 15 octombrie 2023
4,De inchiriat 2 camere - complet mobilat - Aqua...,600.0,https://www.storia.ro/ro/oferta/de-inchiriat-2...,0,60.0,"Bucuresti, Sectorul 6",Azi la 15:38
5,Apartament 2 camere Lux zona Politehnica/Lujer...,499.0,https://www.storia.ro/ro/oferta/apartament-2-c...,0,58.0,"Bucuresti, Sectorul 6",Azi la 15:29
6,Lujerului | 2 Camere | Centrala | Loc De Parc...,570.0,https://www.storia.ro/ro/oferta/lujerului-2-ca...,0,62.0,"Bucuresti, Sectorul 6",Azi la 15:17
7,Închiriez apartament,201.0,/d/oferta/inchiriez-apartament-IDfRSpR.html,0,67.0,"Bucuresti, Sectorul 6",Azi la 15:03
8,Inchiriez Apartament 3 camere Drumul Taberei,353.0,/d/oferta/inchiriez-apartament-3-camere-drumul...,0,60.0,"Bucuresti, Sectorul 6",Azi la 14:53
9,Grozavesti | Regie Residence | Centrala | Ac |...,550.0,https://www.storia.ro/ro/oferta/grozavesti-reg...,0,50.0,"Bucuresti, Sectorul 6",Azi la 14:36


In [47]:
df = df.dropna()

In [49]:
df.head()

,title,price,link,negotiable,surface,location,date
1,"Particular, ofer spre inchiriere apartament 2 ...",450.0,/d/oferta/particular-ofer-spre-inchiriere-apar...,0,50.0,"Bucuresti, Sectorul 6",15 octombrie 2023
2,"Apartament 2 camere Plaza Romania, Lujerului, ...",620.0,/d/oferta/apartament-2-camere-plaza-romania-lu...,1,53.0,"Bucuresti, Sectorul 6",Reactualizat la 15 octombrie 2023
3,2 Camere | Lujerului | 57 MP | Decomandat | 3 ...,500.0,/d/oferta/2-camere-lujerului-57-mp-decomandat-...,0,60.0,"Bucuresti, Sectorul 6",Reactualizat la 15 octombrie 2023
4,De inchiriat 2 camere - complet mobilat - Aqua...,600.0,https://www.storia.ro/ro/oferta/de-inchiriat-2...,0,60.0,"Bucuresti, Sectorul 6",Azi la 15:38
5,Apartament 2 camere Lux zona Politehnica/Lujer...,499.0,https://www.storia.ro/ro/oferta/apartament-2-c...,0,58.0,"Bucuresti, Sectorul 6",Azi la 15:29
